In [1]:
from es import DocumentStore, Document

In [2]:
host = 'http://172.31.18.19:9200'

In [3]:
index = 'tmp_es_template'

In [4]:
storeclient = DocumentStore(host)

In [5]:
storeclient.create_store(index, lang='en', reset_index=True)

In [6]:
with open('sample.txt') as f:
    data = [Document(text=s.strip(), id=i) for i, s in enumerate(f.readlines())]

In [7]:
data

[Document(text='machine learning', id=0),
 Document(text='deep learning', id=1),
 Document(text='organizational behavior', id=2),
 Document(text='big disappointment', id=3)]

In [8]:
storeclient.add_documents(index=index, documents=data)

In [9]:
query = 'machine learn'

In [10]:
storeclient.search_by_word_matching(index, query)

[{'text': 'machine learning',
  'score': 1.89712,
  'highlight': ['<b>machine</b> <b>learning</b>']},
 {'text': 'deep learning',
  'score': 0.6931471,
  'highlight': ['deep <b>learning</b>']}]

In [12]:
query_string = "organzation"

query = {
    "query": { 
        "multi_match": {
            "query": "*%s*" % query_string,
            "fields": [
                "text"
            ]
        }
    },
    "suggest" : {
        "query_suggestion" : {
            "text" : query_string,
            "term" : {
                "field" : "text"
            }
        }
    },
    "_source": ["text"],
    "highlight": {
        "pre_tags": ["<b>"], 
        "post_tags": ["</b>"], 
        "fields": {    
            "text": { "fragment_size": 200 }
        }
    }
}
res = storeclient._es.search(index=index, body=query)
res

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'suggest': {'query_suggestion': [{'text': 'organz',
    'offset': 0,
    'length': 11,
    'options': [{'text': 'organiz', 'score': 0.8333333, 'freq': 1}]}]}}

In [38]:
res

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.9331132,
  'hits': [{'_index': 'tmp_es_template',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.9331132,
    '_source': {'text': "c'est quoi machine learning"},
    'highlight': {'text': ["c'est <b>quoi</b> machine learning"]}}]},
 'suggest': {'query_suggestion': [{'text': 'quoi',
    'offset': 0,
    'length': 5,
    'options': []}]}}